In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
import os
import warnings
import pickle

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0=all, 1=info, 2=warning, 3=error
warnings.filterwarnings('ignore', category=UserWarning)

np.random.seed(42)
tf.random.set_seed(42)

physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

def load_data(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))
    df = df.set_index('date')
    return df

def create_forecast_sequences(data, seq_length, forecast_horizon):
    X, y = [], []
    for i in range(len(data) - seq_length - forecast_horizon + 1):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length:i+seq_length+forecast_horizon])
    return np.array(X), np.array(y)

def calculate_metrics(y_true, y_pred, y_lower, y_upper):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    y_lower = y_lower.flatten()
    y_upper = y_upper.flatten()
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    within_interval = np.logical_and(y_true >= y_lower, y_true <= y_upper)
    picp = np.mean(within_interval)
    mpiw = np.mean(y_upper - y_lower)
    
    return rmse, picp, mpiw

def create_train_model(X_train, y_train, forecast_horizon, n_features, model_dir, bootstrap_id):
    os.makedirs(model_dir, exist_ok=True)
    
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], n_features)),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(forecast_horizon * n_features)
    ])

    model.compile(optimizer='adam', loss='mse')

    checkpoint_path = os.path.join(model_dir, f"pca_model_{bootstrap_id}.keras")
    
    checkpoint = ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        save_weights_only=False,
        verbose=0
    )
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=0
    )

    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[checkpoint, early_stopping],
        verbose=0
    )

    return model, history

def generate_bootstrap_forecasts(X_train, y_train, X_val, forecast_horizon, n_components, 
                                num_bootstraps=10, save_results=True, model_dir="", results_dir=""):
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)
    
    forecasts = []
    models = []

    for i in range(num_bootstraps):
        indices = np.random.choice(len(X_train), len(X_train), replace=True)
        X_bootstrap = X_train[indices]
        y_bootstrap = y_train[indices]

        model, _ = create_train_model(X_bootstrap, y_bootstrap, forecast_horizon, n_components, model_dir, i)
        models.append(model)

        forecast = model.predict(X_val, verbose=0)
        forecasts.append(forecast)

    forecasts = np.array(forecasts)

    mean_forecast = np.mean(forecasts, axis=0)
    lower_bound = np.quantile(forecasts, 0.05, axis=0)
    upper_bound = np.quantile(forecasts, 0.95, axis=0)

    if save_results:
        results = {
            'mean_forecast': mean_forecast,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound,
            'forecasts': forecasts
        }
        
        results_path = os.path.join(results_dir, 'pca_forecast_results.pkl')
        with open(results_path, 'wb') as f:
            pickle.dump(results, f)

    return mean_forecast, lower_bound, upper_bound, models, forecasts

def load_forecast_results(results_path):
    if not os.path.exists(results_path):
        raise FileNotFoundError(f"Results file not found at {results_path}")
    
    with open(results_path, 'rb') as f:
        results = pickle.load(f)
    
    return results['mean_forecast'], results['lower_bound'], results['upper_bound'], results['forecasts']

def load_trained_models(num_bootstraps, model_dir):
    if not os.path.exists(model_dir):
        raise FileNotFoundError(f"Model directory {model_dir} not found")
    
    models = []
    for i in range(num_bootstraps):
        model_path = os.path.join(model_dir, f"pca_model_{i}.keras")
        if os.path.exists(model_path):
            model = load_model(model_path)
            models.append(model)
    
    return models

def predict_future(model, last_sequence, future_steps, forecast_horizon, pca_scaler, pca, n_components, model_id):
    future_predictions_pc = []
    curr_sequence = last_sequence.copy()

    steps_completed = 0
    while steps_completed < future_steps:
        curr_sequence_reshaped = curr_sequence.reshape(1, curr_sequence.shape[0], curr_sequence.shape[1])
        next_preds = model.predict(curr_sequence_reshaped, verbose=0)[0]
        
        next_preds_reshaped = next_preds.reshape(forecast_horizon, n_components)
        
        steps_to_add = min(forecast_horizon, future_steps - steps_completed)
        future_predictions_pc.extend(next_preds_reshaped[:steps_to_add])
        
        curr_sequence = np.roll(curr_sequence, -steps_to_add, axis=0)
        
        for i in range(steps_to_add):
            curr_sequence[-steps_to_add+i] = next_preds_reshaped[i]
        
        steps_completed += steps_to_add

    future_predictions_pc = np.array(future_predictions_pc)
    
    future_predictions_scaled = pca_scaler.inverse_transform(future_predictions_pc)
    
    future_predictions_original = pca.inverse_transform(future_predictions_scaled)
    
    future_predictions_ssn = future_predictions_original[:, 0].reshape(-1, 1)
    future_predictions_tsi = future_predictions_original[:, 1].reshape(-1, 1)

    return future_predictions_ssn, future_predictions_tsi, future_predictions_pc

def generate_future_dates(last_date, num_months):
    future_dates = pd.date_range(start=last_date, periods=num_months+1, freq='MS')[1:]
    return future_dates

def calculate_evaluation_metrics(actual_values, mean_forecast_inv, lower_bound_inv, upper_bound_inv):
    rmse_values = []
    picp_values = []
    mpiw_values = []

    for i in range(len(actual_values)):
        rmse, picp, mpiw = calculate_metrics(
            actual_values[i],
            mean_forecast_inv[i],
            lower_bound_inv[i],
            upper_bound_inv[i]
        )
        rmse_values.append(rmse)
        picp_values.append(picp)
        mpiw_values.append(mpiw)

    return np.mean(rmse_values), np.mean(picp_values), np.mean(mpiw_values)

def plot_forecast(df, val_dates, mean_forecast_inv, lower_bound_inv, upper_bound_inv, 
                 future_dates, future_mean, future_lower, future_upper,
                 cycle_24_start, cycle_25_start, forecast_horizon, target_var, output_path):
    start_date = pd.to_datetime('1996-01-01')
    plt.figure(figsize=(15, 8))

    historical_mask = df.index >= start_date
    plt.plot(df.index[historical_mask], df[target_var][historical_mask], 'b-', label=f'Historical {target_var} Data')

    for i in range(len(mean_forecast_inv)):
        pred_dates = val_dates[i:i+forecast_horizon]
        if len(pred_dates) == len(mean_forecast_inv[i]):
            plt.plot(pred_dates, mean_forecast_inv[i], 'r-', alpha=0.5, label=f'Validation {target_var} Predictions' if i == 0 else "")
            plt.fill_between(pred_dates, lower_bound_inv[i], upper_bound_inv[i], color='r', alpha=0.2, label='90% Confidence Interval' if i == 0 else "")

    plt.plot(future_dates, future_mean, 'g-', label=f'Future {target_var} Predictions')
    plt.fill_between(future_dates, future_lower.flatten(), future_upper.flatten(), color='g', alpha=0.2, label='Future 90% Confidence Interval')
    plt.axvline(x=cycle_24_start, color='k', linestyle='--', label='Cycle 24 Start')
    plt.axvline(x=cycle_25_start, color='k', linestyle='--', label='Cycle 25 Start')
    plt.title(f'{target_var} Forecast (Seq: {seq_length//12} years, Horizon: {forecast_horizon} months)')
    plt.xlabel('Date')
    plt.ylabel(f'{target_var}')
    plt.legend()
    plt.grid(True)
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

def plot_principal_components(pc_data, pc_dates, future_pc_data, future_dates, n_components, output_path):
    plt.figure(figsize=(15, 8))
    
    for i in range(n_components):
        plt.subplot(n_components, 1, i+1)
        plt.plot(pc_dates, pc_data[:, i], 'b-', label=f'Historical PC{i+1}')
        plt.plot(future_dates, future_pc_data[:, i], 'g-', label=f'Forecast PC{i+1}')
        plt.title(f'Principal Component {i+1}')
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

if __name__ == "__main__":
    TRAIN_MODELS = True
    
    ssn_df = load_data('/Users/suryatejachalla/Projects/Solar-Data-Analysis/Data/processed/sunspot.csv')
    tsi_df = load_data('/Users/suryatejachalla/Projects/Solar-Data-Analysis/Data/processed/tsi.csv')
    
    df = pd.merge(ssn_df, tsi_df, left_index=True, right_index=True, suffixes=('', '_tsi'))
    df = df.rename(columns={'tsi_tsi': 'tsi'})
    
    print("Data summary:")
    print(df.describe())
    print(f"Correlation between SSN and TSI: {df['ssn'].corr(df['tsi']):.4f}")
    
    features = df[['ssn', 'tsi']].values
    
    n_components = 2
    pca = PCA(n_components=n_components)
    
    pca_features = pca.fit_transform(features)
    
    print("PCA explained variance ratio:", pca.explained_variance_ratio_)
    
    pca_scaler = MinMaxScaler(feature_range=(0, 1))
    pca_scaled = pca_scaler.fit_transform(pca_features)
    
    with open('pca_model.pkl', 'wb') as f:
        pickle.dump(pca, f)
    with open('pca_scaler.pkl', 'wb') as f:
        pickle.dump(pca_scaler, f)
    
    cycle_24_start = pd.to_datetime('2008-12-01')
    cycle_25_start = pd.to_datetime('2019-12-01')
    validation_start = pd.to_datetime('2021-01-01')
    
    train_data = pca_scaled[df.index < cycle_25_start]
    val_data = pca_scaled[(df.index >= validation_start)]
    val_dates = df.index[(df.index >= validation_start)]
    
    sequence_years = range(6, 12)
    forecast_months = [1, 3, 6]
    
    num_bootstraps = 10
    
    last_date = df.index[-1]
    future_steps = (2030 - last_date.year) * 12 + (12 - last_date.month)
    future_dates = generate_future_dates(last_date, future_steps)
    
    print(f"Total number of combinations: {len(sequence_years) * len(forecast_months)}")
    
    base_output_dir = "solar_forecast_combinations"
    os.makedirs(base_output_dir, exist_ok=True)
    
    ssn_plots_dir = os.path.join(base_output_dir, "ssn_plots")
    tsi_plots_dir = os.path.join(base_output_dir, "tsi_plots")
    os.makedirs(ssn_plots_dir, exist_ok=True)
    os.makedirs(tsi_plots_dir, exist_ok=True)
    
    best_combination = None
    best_picp_ssn = 0
    best_picp_tsi = 0
    
    for seq_years in sequence_years:
        for fh in forecast_months:
            seq_length = seq_years * 12
            forecast_horizon = fh
            
            combo_name = f"seq{seq_years}y_fh{forecast_horizon}m"
            print(f"\nProcessing combination: {combo_name}")
            
            model_dir = os.path.join(base_output_dir, "models", combo_name)
            results_dir = os.path.join(base_output_dir, "results", combo_name)
            
            os.makedirs(model_dir, exist_ok=True)
            os.makedirs(results_dir, exist_ok=True)
            
            X_train, y_train = create_forecast_sequences(train_data, seq_length, forecast_horizon)
            y_train_flattened = y_train.reshape(y_train.shape[0], -1)
            
            X_val = []
            y_val_pc = []
            
            for i in range(len(val_dates) - forecast_horizon + 1):
                start_idx = len(df.index[df.index < val_dates[i]]) - seq_length
                end_idx = len(df.index[df.index < val_dates[i]])
                
                if start_idx >= 0 and end_idx < len(pca_scaled):
                    X_val.append(pca_scaled[start_idx:end_idx])
                    y_val_pc.append(pca_scaled[end_idx:end_idx+forecast_horizon])
            
            if not X_val:
                print(f"Skipping {combo_name}: Insufficient validation data")
                continue
                
            X_val = np.array(X_val)
            y_val_pc = np.array(y_val_pc)
            
            if TRAIN_MODELS:
                mean_forecast, lower_bound, upper_bound, models, forecasts = generate_bootstrap_forecasts(
                    X_train, y_train_flattened, X_val, forecast_horizon, n_components, 
                    num_bootstraps=num_bootstraps, model_dir=model_dir, results_dir=results_dir
                )
            else:
                results_path = os.path.join(results_dir, 'pca_forecast_results.pkl')
                mean_forecast, lower_bound, upper_bound, forecasts = load_forecast_results(results_path)
                models = load_trained_models(num_bootstraps, model_dir)
            
            mean_forecast_reshaped = mean_forecast.reshape(-1, forecast_horizon, n_components)
            lower_bound_reshaped = lower_bound.reshape(-1, forecast_horizon, n_components)
            upper_bound_reshaped = upper_bound.reshape(-1, forecast_horizon, n_components)
            
            mean_forecast_scaled = np.array([pca_scaler.inverse_transform(f) for f in mean_forecast_reshaped])
            lower_bound_scaled = np.array([pca_scaler.inverse_transform(f) for f in lower_bound_reshaped])
            upper_bound_scaled = np.array([pca_scaler.inverse_transform(f) for f in upper_bound_reshaped])
            
            mean_forecast_original = np.array([pca.inverse_transform(f) for f in mean_forecast_scaled])
            lower_bound_original = np.array([pca.inverse_transform(f) for f in lower_bound_scaled])
            upper_bound_original = np.array([pca.inverse_transform(f) for f in upper_bound_scaled])
            
            ssn_mean_forecast_inv = np.array([f[:, 0] for f in mean_forecast_original])
            ssn_lower_bound_inv = np.array([f[:, 0] for f in lower_bound_original])
            ssn_upper_bound_inv = np.array([f[:, 0] for f in upper_bound_original])
            
            tsi_mean_forecast_inv = np.array([f[:, 1] for f in mean_forecast_original])
            tsi_lower_bound_inv = np.array([f[:, 1] for f in lower_bound_original])
            tsi_upper_bound_inv = np.array([f[:, 1] for f in upper_bound_original])
            
            ssn_actual_values = []
            tsi_actual_values = []
            
            for i in range(len(y_val_pc)):
                pc_inv = pca_scaler.inverse_transform(y_val_pc[i])
                original_inv = pca.inverse_transform(pc_inv)
                
                ssn_inv = original_inv[:, 0]
                tsi_inv = original_inv[:, 1]
                
                ssn_actual_values.append(ssn_inv)
                tsi_actual_values.append(tsi_inv)
            
            ssn_actual_values = np.array(ssn_actual_values)
            tsi_actual_values = np.array(tsi_actual_values)
            
            ssn_avg_rmse, ssn_avg_picp, ssn_avg_mpiw = calculate_evaluation_metrics(
                ssn_actual_values, ssn_mean_forecast_inv, ssn_lower_bound_inv, ssn_upper_bound_inv
            )
            
            tsi_avg_rmse, tsi_avg_picp, tsi_avg_mpiw = calculate_evaluation_metrics(
                tsi_actual_values, tsi_mean_forecast_inv, tsi_lower_bound_inv, tsi_upper_bound_inv
            )
            
            print(f"SSN - RMSE: {ssn_avg_rmse:.4f}, PICP: {ssn_avg_picp:.4f}, MPIW: {ssn_avg_mpiw:.4f}")
            print(f"TSI - RMSE: {tsi_avg_rmse:.4f}, PICP: {tsi_avg_picp:.4f}, MPIW: {tsi_avg_mpiw:.4f}")
            
            avg_picp = (ssn_avg_picp + tsi_avg_picp) / 2
            if best_combination is None or avg_picp > (best_picp_ssn + best_picp_tsi) / 2:
                best_combination = combo_name
                best_picp_ssn = ssn_avg_picp
                best_picp_tsi = tsi_avg_picp
            
            last_sequence = pca_scaled[-seq_length:].reshape(seq_length, n_components)
            
            future_ssn_forecasts = []
            future_tsi_forecasts = []
            future_pc_forecasts = []
            
            for i, model in enumerate(models):
                future_ssn, future_tsi, future_pc = predict_future(
                    model, last_sequence, future_steps, forecast_horizon, 
                    pca_scaler, pca, n_components, i
                )
                future_ssn_forecasts.append(future_ssn)
                future_tsi_forecasts.append(future_tsi)
                future_pc_forecasts.append(future_pc)
            
            future_ssn_forecasts = np.array(future_ssn_forecasts)
            future_tsi_forecasts = np.array(future_tsi_forecasts)
            future_pc_forecasts = np.array(future_pc_forecasts)
            
            future_ssn_mean = np.mean(future_ssn_forecasts, axis=0)
            future_ssn_lower = np.quantile(future_ssn_forecasts, 0.05, axis=0)
            future_ssn_upper = np.quantile(future_ssn_forecasts, 0.95, axis=0)
            
            future_tsi_mean = np.mean(future_tsi_forecasts, axis=0)
            future_tsi_lower = np.quantile(future_tsi_forecasts, 0.05, axis=0)
            future_tsi_upper = np.quantile(future_tsi_forecasts, 0.95, axis=0)
            
            future_pc_mean = np.mean(future_pc_forecasts, axis=0)
            
            future_results = {
                'future_ssn_mean': future_ssn_mean,
                'future_ssn_lower': future_ssn_lower,
                'future_ssn_upper': future_ssn_upper,
                'future_tsi_mean': future_tsi_mean,
                'future_tsi_lower': future_tsi_lower,
                'future_tsi_upper': future_tsi_upper,
                'future_pc_mean': future_pc_mean,
                'future_dates': future_dates,
                'metrics': {
                    'ssn_rmse': ssn_avg_rmse,
                    'ssn_picp': ssn_avg_picp,
                    'ssn_mpiw': ssn_avg_mpiw,
                    'tsi_rmse': tsi_avg_rmse,
                    'tsi_picp': tsi_avg_picp,
                    'tsi_mpiw': tsi_avg_mpiw
                }
            }
            
            with open(os.path.join(results_dir, 'pca_future_predictions.pkl'), 'wb') as f:
                pickle.dump(future_results, f)
            
            ssn_plot_path = os.path.join(ssn_plots_dir, f"{combo_name}_ssn_forecast.png")
            tsi_plot_path = os.path.join(tsi_plots_dir, f"{combo_name}_tsi_forecast.png")
            pc_plot_path = os.path.join(results_dir, f"{combo_name}_principal_components.png")
            
            plot_forecast(
                df, val_dates, ssn_mean_forecast_inv, ssn_lower_bound_inv, ssn_upper_bound_inv,
                future_dates, future_ssn_mean, future_ssn_lower, future_ssn_upper,
                cycle_24_start, cycle_25_start, forecast_horizon, 'ssn', ssn_plot_path
            )
            
            plot_forecast(
                df, val_dates, tsi_mean_forecast_inv, tsi_lower_bound_inv, tsi_upper_bound_inv,
                future_dates, future_tsi_mean, future_tsi_lower, future_tsi_upper,
                cycle_24_start, cycle_25_start, forecast_horizon, 'tsi', tsi_plot_path
            )
            
            plot_principal_components(
                pca_scaled, df.index, future_pc_mean, future_dates, n_components, pc_plot_path
            )
    
    print(f"\nBest combination: {best_combination}")
    print(f"Best SSN PICP: {best_picp_ssn:.4f}")
    print(f"Best TSI PICP: {best_picp_tsi:.4f}")
    print(f"Average PICP: {(best_picp_ssn + best_picp_tsi) / 2:.4f}")
    print("PCA-based forecasting process completed successfully")

Data summary:
              ssn         year       month     year_tsi   month_tsi  \
count  924.000000   924.000000  924.000000   924.000000  924.000000   
mean    93.807468  1985.583333    6.500000  1985.583333    6.500000   
std     71.508209    22.243618    3.453922    22.243618    3.453922   
min      1.800000  1947.000000    1.000000  1947.000000    1.000000   
25%     28.325000  1966.000000    3.750000  1966.000000    3.750000   
50%     84.050000  1986.000000    6.500000  1986.000000    6.500000   
75%    149.950000  2005.000000    9.250000  2005.000000    9.250000   
max    285.000000  2024.000000   12.000000  2024.000000   12.000000   

               tsi  
count   924.000000  
mean   1363.423637  
std       0.354047  
min    1362.889401  
25%    1363.075220  
50%    1363.427315  
75%    1363.741120  
max    1364.138153  
Correlation between SSN and TSI: 0.9065
PCA explained variance ratio: [9.99995630e-01 4.37041416e-06]
Total number of combinations: 18

Processing combinatio